此檔案進行包含 Dropout 的 Swin Transformer 訓練（針對全部資料集，無 Validation）。主要流程包括：

1. **模型架構**：採用 Swin Transformer Base (224×224) 作為基底模型，並在最後的全連接層加入 Dropout（p=0.5）以防止 Overfitting。使用 `timm` 庫中的預訓練模型，並設置選項來凍結部分層（如 `layers.0` 和 `layers.1`）以提升訓練效率。

2. **數據處理與增強**：
   - 使用 `datasets.py` 中的 `MultiLabelDataset` 類別來處理標籤為多個類別的圖像數據。  
   - 應用多種數據增強技術，如隨機裁剪、旋轉、色彩抖動等，來提高模型的泛化能力。  
   - 使用 Mixup 技術，將不同樣本進行加權混合。

3. **訓練配置**：
   - 使用 `BCEWithLogitsLoss` 作為損失函數來處理多標籤問題，並配置 Adam Optimizer。  
   - 設置學習率調度器（Cosine Annealing）來動態調整學習率。

4. **評估指標**：訓練過程中計算 F1-score、Precision、Recall、MAP 等指標。

5. **模型儲存與視覺化**：
   - 每個訓練週期（Epoch）後保存模型參數，並記錄訓練損失與指標。  
   - 使用 TensorBoard 來可視化訓練過程中的損失和指標變化，方便分析模型學習情況。

In [ ]:
import timm
print(timm.list_models('swin*', pretrained=True))

['swin_base_patch4_window7_224.ms_in1k', 'swin_base_patch4_window7_224.ms_in22k', 'swin_base_patch4_window7_224.ms_in22k_ft_in1k', 'swin_base_patch4_window12_384.ms_in1k', 'swin_base_patch4_window12_384.ms_in22k', 'swin_base_patch4_window12_384.ms_in22k_ft_in1k', 'swin_large_patch4_window7_224.ms_in22k', 'swin_large_patch4_window7_224.ms_in22k_ft_in1k', 'swin_large_patch4_window12_384.ms_in22k', 'swin_large_patch4_window12_384.ms_in22k_ft_in1k', 'swin_s3_base_224.ms_in1k', 'swin_s3_small_224.ms_in1k', 'swin_s3_tiny_224.ms_in1k', 'swin_small_patch4_window7_224.ms_in1k', 'swin_small_patch4_window7_224.ms_in22k', 'swin_small_patch4_window7_224.ms_in22k_ft_in1k', 'swin_tiny_patch4_window7_224.ms_in1k', 'swin_tiny_patch4_window7_224.ms_in22k', 'swin_tiny_patch4_window7_224.ms_in22k_ft_in1k', 'swinv2_base_window8_256.ms_in1k', 'swinv2_base_window12_192.ms_in22k', 'swinv2_base_window12to16_192to256.ms_in22k_ft_in1k', 'swinv2_base_window12to24_192to384.ms_in22k_ft_in1k', 'swinv2_base_window16_

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.cuda.amp import GradScaler, autocast
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from PIL import Image

from timm import create_model

from datasets import MultiLabelDataset

import torch.nn.functional as F

class MultiLabelSwinTransformer(nn.Module):
    def __init__(self, num_classes, pretrained=True, freeze_layers=False):
        super(MultiLabelSwinTransformer, self).__init__()
        # 使用 Swin Base (224×224)
        self.model = create_model(
            'swin_base_patch4_window7_224',
            pretrained=pretrained
        )
        in_features = self.model.head.in_features
        self.model.head = nn.Identity()

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(in_features, num_classes)
        )

        if freeze_layers:
            for name, param in self.model.named_parameters():
                if "layers.0" in name or "layers.1" in name:
                    param.requires_grad = False

    def forward(self, x):
        # x shape: [batch_size, 3, 224, 224]

        x = self.model.forward_features(x)
        # 此時 x shape: [batch_size, 7, 7, 1024]

        x = x.permute(0, 3, 1, 2)
        # 現在 x shape: [batch_size, 1024, 7, 7]

        x = F.adaptive_avg_pool2d(x, (1, 1))
        # shape: [batch_size, 1024, 1, 1]

        x = x.flatten(1)
        # shape: [batch_size, 1024]

        x = self.classifier(x)
        return x

def calculate_metrics(y_true, y_pred, y_scores):

    from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, average_precision_score

    metrics = {}
    metrics['f1'] = f1_score(y_true, y_pred, average='micro', zero_division=0)
    metrics['precision'] = precision_score(y_true, y_pred, average='micro', zero_division=0)
    metrics['recall'] = recall_score(y_true, y_pred, average='micro', zero_division=0)

    try:
        metrics['roc_auc'] = roc_auc_score(y_true, y_pred, average='micro')
    except:
        metrics['roc_auc'] = 0.0

    try:
        ap_per_class = average_precision_score(y_true, y_scores, average=None)
        metrics['map'] = np.mean(ap_per_class)
    except Exception as e:
        print("Error calculating MAP:", e)
        metrics['map'] = 0.0

    return metrics

def mixup_data(x, y, alpha=1.0):
    """
    x: [batch_size, C, H, W]
    y: [batch_size, num_classes]
    alpha: default=1.0
    Returns:
        mixed_x, y_a, y_b, lam
    """
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1.0

    batch_size = x.size(0)
    if torch.cuda.is_available():
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    """
    pred: [batch_size, num_classes] (logits)
    y_a, y_b: [batch_size, num_classes]
    lam: mixup ratio
    """
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)



def main():
    with open('class_mapping.json', 'r', encoding='utf-8') as f:
        class_mapping = json.load(f)
    num_classes = len(class_mapping)

    ground_truth = pd.read_csv('ground_truth_probabilities.csv')
    if 'filename' not in ground_truth.columns:
        raise ValueError("CSV 文件缺少 'filename' 欄位，請檢查。")

    print(f"Number of training samples: {len(ground_truth)}")

    train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])

    img_dir = 'train_data'
    train_dataset = MultiLabelDataset(
        dataframe=ground_truth,
        img_dir=img_dir,
        class_mapping=class_mapping,
        transform=train_transforms
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=64,
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    # freeze_layers=False
    model = MultiLabelSwinTransformer(
        num_classes=num_classes,
        pretrained=True,
        freeze_layers=False
    ).to(device)

    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

    scaler = GradScaler()
    writer = SummaryWriter('runs/multi_label_experiment_swin_no_freeze')

    num_epochs = 80

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 30)

        model.train()
        running_loss = 0.0
        all_preds = []
        all_labels = []

        for inputs, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            # Mixup
            inputs, labels_a, labels_b, lam = mixup_data(inputs, labels, alpha=1.0)

            with autocast():
                outputs = model(inputs)  # shape [batch_size, num_classes]
                loss = mixup_criterion(criterion, outputs, labels_a, labels_b, lam)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item() * inputs.size(0)

            preds = torch.sigmoid(outputs).detach().cpu().numpy()  # (batch_size, num_classes)
            all_preds.append(preds)
            all_labels.append(labels.detach().cpu().numpy())

        epoch_loss = running_loss / len(train_loader.dataset)

        all_preds = np.vstack(all_preds)   # shape (N, num_classes)
        all_labels = np.vstack(all_labels) # shape (N, num_classes)

        preds_binary = (all_preds > 0.5).astype(int)
        train_metrics = calculate_metrics(all_labels, preds_binary, all_preds)

        print(
            f"Training Loss: {epoch_loss:.4f} | "
            f"F1: {train_metrics['f1']:.4f} | "
            f"MAP: {train_metrics['map']:.4f}"
        )

        model_path = f"model_epoch_{epoch+1}_swin_no_freeze.pth"
        torch.save(model.state_dict(), model_path)
        print(f"Model for epoch {epoch+1} saved as {model_path}.")

        scheduler.step()

        writer.add_scalar('Loss/train', epoch_loss, epoch)
        writer.add_scalar('F1/train', train_metrics['f1'], epoch)
        writer.add_scalar('mAP/train', train_metrics['map'], epoch)

    torch.save(model.state_dict(), 'final_model_swin_transformer_no_freeze.pth')
    print("Final model saved as 'final_model_swin_transformer_no_freeze.pth'.")

    writer.close()

if __name__ == '__main__':
    main()


Number of training samples: 37866
Using device: cuda


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-3-4b9f567871ef>:201: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()



Epoch 1/80
------------------------------


Training Epoch 1:   0%|          | 0/592 [00:00<?, ?it/s]<ipython-input-3-4b9f567871ef>:224: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Training Epoch 1: 100%|██████████| 592/592 [11:00<00:00,  1.12s/it]


Training Loss: 0.0898 | F1: 0.1092 | MAP: 0.0447
Model for epoch 1 saved as model_epoch_1_swin_no_freeze.pth.

Epoch 2/80
------------------------------


...

Training Loss: 0.0426 | F1: 0.4558 | MAP: 0.3565
Model for epoch 78 saved as model_epoch_78_swin_no_freeze.pth.

Epoch 79/80
------------------------------


Training Epoch 79:   0%|          | 0/592 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-3-4b9f567871ef>:224: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Training Epoch 79: 100%|██████████| 592/592 [10:22<00:00,  1.05s/it]


Training Loss: 0.0434 | F1: 0.4353 | MAP: 0.3207
Model for epoch 79 saved as model_epoch_79_swin_no_freeze.pth.

Epoch 80/80
------------------------------


Training Epoch 80:   0%|          | 0/592 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-3-4b9f567871ef>:224: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Training Epoch 80: 100%|██████████| 592/592 [10:22<00:00,  1.05s/it]


Training Loss: 0.0437 | F1: 0.4268 | MAP: 0.3157
Model for epoch 80 saved as model_epoch_80_swin_no_freeze.pth.
Final model saved as 'final_model_swin_transformer_no_freeze.pth'.
